<a href="https://colab.research.google.com/github/AlexanderBelfort/MDPRKT/blob/main/KatiePretrainedHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HuggingFace model implementation of Katie's Transcript Data**

In [1]:
!pip install transformers

In [2]:
from transformers import pipeline

**Question Answering from HuggingFace**

1.   Load the data
2.   Test the data



In [3]:
with open ("1.txt", "r") as myfile:
  next(myfile)
  data=myfile.read().replace('\n', '')

# data

**Ibuprofen takes 30 minutes to work** is the actual answer given precisly by **Allen**. This is a weird return.

In [4]:
nlp = pipeline('question-answering')
nlp({
    'question': 'How long does ibuprofen take to work?',
    'context': data
})

{'answer': 'two months',
 'end': 1003,
 'score': 0.7775351405143738,
 'start': 993}

In [5]:
# TESTING
# iteration does not work hm

x = ["How long does ibuprofen take to work?",\
     "When does the pain come?",\
     "Should they send the patient the questonnaire by post or email?",\
     "What is medical management?",\
     "What is the name of the surgery?",\
     "What does the computer do?"]

y = [item.strip() for item in x]

def testing(theList):
  nlp = pipeline('question-answering')

  for each_element in theList:
    nlp({'question': each_element,'context': data})


  

testing(y)

# does not return anything for some weird reason

In [6]:
nlp = pipeline('question-answering')
nlp({
    'question': 'When does the pain come?',
    'context': data
})

# in the night time is the result from allennlp so it's good

{'answer': 'night time', 'end': 588, 'score': 0.1645297110080719, 'start': 578}

This however, is a very **bad return as it returns the question itself**. AllenNLP returns the actual answer **"post"**.

In [7]:
nlp = pipeline('question-answering')
nlp({
    'question': 'Should they send the patient the questonnaire by post or email?',
    'context': data
})


{'answer': 'send you the study questionnaires by post, or by email',
 'end': 14246,
 'score': 0.1665525585412979,
 'start': 14192}

**AllenNLP returns the same value more or less.**

In [8]:
nlp({
    'question': 'What is medical management?',
    'context': data
})


{'answer': 'dietary advice and simple painkillers',
 'end': 2114,
 'score': 0.7215187549591064,
 'start': 2077}

**AllenNLP returns P2 - ...the surgery is normally keyhole**, but the correct precise answer containing the actual medical term is outlined in this model here.

In [9]:
nlp({
    'question': 'What is the name of the surgery?',
    'context': data
})


{'answer': 'laparoscopic cholecystectomy',
 'end': 2013,
 'score': 0.6391268372535706,
 'start': 1985}

**AllenNLP returns -> if you consent to take part in the study, decides what happens to you - whether you get the surgery, or whether you don’t** which is more or less the same result, but the returned answer in this model is more **precise**

In [10]:
nlp({
    'question': 'What does the computer do?',
    'context': data
})

{'answer': 'decides what happens to you',
 'end': 11956,
 'score': 0.5366376638412476,
 'start': 11929}

**Text Generation**

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [12]:
tokeniser = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

**Build our model**

In [13]:
text = "I suffer from gallbladder"
indexed_tokens = tokeniser.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
model.eval()
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

**Test our model.**

In [14]:
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

print(outputs[0].shape)

predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokeniser.decode(indexed_tokens + [predicted_index])

print(predicted_text)

torch.Size([1, 6, 50257])
I suffer from gallbladder problems


**Test text completion**

In [15]:
chars = 0
text = "How often does gallbladder disease"
while chars<50:
  chars += 1
  indexed_tokens = tokeniser.encode(text)
  tokens_tensors = torch.tensor([indexed_tokens])
  tokens_tensors = tokens_tensors.to('cuda')
#  model = model.to('cuda')
  with torch.no_grad():
    outputs = model(tokens_tensors)
    predictions = outputs[0]
  predicted_index = torch.argmax(predictions[0,-1,:]).item()
  text = tokeniser.decode(indexed_tokens + [predicted_index])

print(text)

How often does gallbladder disease occur in the United States?


Gallbladder disease is a common and serious condition that affects the lining of the gallbladder. It is caused by a buildup of fluid in the lining of the gallbladder. The lining of the


In [16]:
!git clone https://github.com/huggingface/pytorch-transformers.git

fatal: destination path 'pytorch-transformers' already exists and is not an empty directory.


In [17]:
!python pytorch-transformers/examples/text-generation/run_generation.py \
    --model_type=gpt2 \
    --length=100 \
    --model_name_or_path=gpt2 \

02/02/2021 12:19:32 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
02/02/2021 12:19:41 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=100, model_name_or_path='gpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='', prompt='', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Model prompt >>> I suffer from gallbladder
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
2021-02-02 12:20:33.233957: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
I suffer from gallbladder stiffness."

"They say, 'Hey, you have an eardrum.' It's not viable when you have an eardrum.' But there is not a stall under your door. You can tell from your doctor or an allergist if your doctor's eardrum is right. If you have an other physician's eardrum

**Exceptional text summarization**

In [19]:
###

from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

**Results**

In [21]:
with open ("1.txt", "r") as myfile:
  next(myfile)
  data=myfile.read().replace('\n', '')


inputs = tokenizer.batch_encode_plus([data], max_length=1024, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=False)

for ids in summary_ids:
    short = tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    print(len(data), len(short))
    print(short)

17463 429
Mr [name] had an ultrasound scan and was found to have stones in the gallbladder. At night time he would feel pain, sometimes twice a week, sometimes long. He had been to hospital once before five months ago, without loss of pain. We are doing a study at the moment, whereby people with gallstones like you, are put into the study and then they are randomised as to what treatment they get. If you agree to be part of this study,


**Changed maxlen to 500 chars and created user-friendly print**

In [26]:

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=500, early_stopping=False)

for ids in summary_ids:
    short = tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    print(len(data), len(short))
    print(short.rstrip().replace('.', '.\n'))

17463 783
Mr [name] had an ultrasound scan and was found to have stones in the gallbladder.
 At night time he would feel pain, sometimes twice a week, sometimes long.
 He had been to hospital once before five months ago, without loss of pain.
 We are doing a study at the moment, whereby people with gallstones like you, are put into the study and then they are randomised as to what treatment they get.
 They can either go for surgery which is a laparoscopic cholecystectomy, or they can be observed with best medical management which is dietary advice and simple painkillers like you have been getting.
 If you agree to be part of this study, you would then … If you go down the line of being observed, and things become worse, then we would see you again to look at taking your gallbladders out.

